In [1]:
import pandas as pd
import plotly.express as px
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, TextVectorization, Embedding
from tensorflow.keras.models import Model

from sklearn.preprocessing import LabelEncoder

from typing import List, Tuple

In [2]:
def data_processing(data_path: str) -> Tuple[pd.DataFrame, LabelEncoder, LabelEncoder]:
    """
    Read the input data, and transform into the structure needed for training a model
    
    Inputs:
        data_path:
            A string path to the input Mr Boston Cocktail Dataset
        
    Outputs:
        data:
            A pandas dataframe with all of the needed columns for training
        ingredient_encoder:
            A label encoder to transform ingredient names to numbers and back
        measurement_encoder:
            A label encoder to transform measurement amounts to numbers and back
    """
    
    df = pd.read_csv("mr-boston-flattened.csv")
    
    ingredients = pd.concat([df["ingredient-1"], df["ingredient-2"], df["ingredient-3"], df["ingredient-4"], df["ingredient-5"], df["ingredient-6"]]).unique()

    ingredient_encoder = LabelEncoder()
    ingredient_encoder.fit(ingredients)
    
    df["ingredient-1"] = ingredient_encoder.transform(df["ingredient-1"])
    df["ingredient-2"] = ingredient_encoder.transform(df["ingredient-2"])
    df["ingredient-3"] = ingredient_encoder.transform(df["ingredient-3"])
    df["ingredient-4"] = ingredient_encoder.transform(df["ingredient-4"])
    df["ingredient-5"] = ingredient_encoder.transform(df["ingredient-5"])
    df["ingredient-6"] = ingredient_encoder.transform(df["ingredient-6"])
    
    measurements = pd.concat([df["measurement-1"], df["measurement-2"], df["measurement-3"], df["measurement-4"], df["measurement-5"], df["measurement-6"]]).unique()

    measurement_encoder = LabelEncoder()
    measurement_encoder.fit(measurements)
    
    df["measurement-1"] = measurement_encoder.transform(df["measurement-1"])
    df["measurement-2"] = measurement_encoder.transform(df["measurement-2"])
    df["measurement-3"] = measurement_encoder.transform(df["measurement-3"])
    df["measurement-4"] = measurement_encoder.transform(df["measurement-4"])
    df["measurement-5"] = measurement_encoder.transform(df["measurement-5"])
    df["measurement-6"] = measurement_encoder.transform(df["measurement-6"])
    
    pieces = []
    pieces.append(df[["name"]])

    for i in range(6):
        temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"ingredient-{i+1}"]), columns=[f"I{i+1} - {c}" for c in ingredient_encoder.classes_])
        pieces.append(temp)

        temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"measurement-{i+1}"]), columns=[f"M{i+1} - {c}" for c in measurement_encoder.classes_])
        pieces.append(temp)

    data = pd.concat(pieces, axis=1)
    
    return data, ingredient_encoder, measurement_encoder
    

In [3]:
def prepare_model(data: pd.DataFrame, ingredient_encoder, measurement_encoder) -> Model:
    """
    
    """
    
    textvec = TextVectorization(
        max_tokens=20000,
        output_sequence_length=5,
        pad_to_max_tokens=True
    )

    ##TODO:
    ##Instead of using the names to adapt, use a more general text source
    extra_vocab = list(pd.read_csv("RAW_recipes.csv")["description"])
    name_vocab = list(data["name"])
    textvec.adapt(name_vocab + extra_vocab)

    voc = textvec.get_vocabulary()
    print(f"Found {len(voc)} different words!")
    word_index = dict(zip(voc, range(len(voc))))
    
    path_to_glove_file = "glove.6B.300d.txt"

    embeddings_index = {}
    with open(path_to_glove_file, encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    
    num_tokens = len(voc) + 2
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    
    encoder_inputs = Input(shape=(1,), dtype=tf.string)

    e = textvec(encoder_inputs)

    e = Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        trainable=False,
    )(e)

    encoder_outputs = LSTM(128)(e)

    d = Dense(64)(encoder_outputs)
    d = Dropout(0.25)(d)
    d = Dense(64)(d)
    d = Dropout(0.25)(d)

    i1 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i1_output")(d)
    i2 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i2_output")(d)
    i3 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i3_output")(d)
    i4 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i4_output")(d)
    i5 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i5_output")(d)
    i6 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i6_output")(d)

    m1 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m1_output")(d)
    m2 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m2_output")(d)
    m3 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m3_output")(d)
    m4 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m4_output")(d)
    m5 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m5_output")(d)
    m6 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m6_output")(d)

    decoder_outputs = [i1, m1, i2, m2, i3, m3, i4, m4, i5, m5, i6, m6]
    model = Model(encoder_inputs, decoder_outputs)
        
    losses = {
        "i1_output":"categorical_crossentropy",
        "i2_output":"categorical_crossentropy",
        "i3_output":"categorical_crossentropy",
        "i4_output":"categorical_crossentropy",
        "i5_output":"categorical_crossentropy",
        "i6_output":"categorical_crossentropy",
        "m1_output":"categorical_crossentropy",
        "m2_output":"categorical_crossentropy",
        "m3_output":"categorical_crossentropy",
        "m4_output":"categorical_crossentropy",
        "m5_output":"categorical_crossentropy",
        "m6_output":"categorical_crossentropy",
    }

    optimizer='adam'
    metrics = {
        "i1_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i2_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i3_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i4_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i5_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i6_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m1_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m2_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m3_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m4_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m5_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m6_output":tf.metrics.CategoricalAccuracy(name='acc'),
    }

    model.compile(loss=losses, metrics=metrics, optimizer=optimizer)
    
    return model

In [4]:
def fit_model(model, data, ingredient_encoder, measurement_encoder, epochs: int = 100) -> None:
    """
    
    """
    
    X = data[["name"]].values

    Ys = {}

    for i in range(6):
        temp = data[[col for col in data.columns if f"I{i+1} - " in col]]
        Ys[f"i{i+1}_output"] = temp.values
        if temp.values.shape[1] != len(ingredient_encoder.classes_):
            raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")

        temp = data[[col for col in data.columns if f"M{i+1} - " in col]]
        Ys[f"m{i+1}_output"] = temp.values
        if temp.values.shape[1] != len(measurement_encoder.classes_):
            raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")

    model.fit(X, Ys, epochs=epochs, verbose=1)

In [5]:
def create_recipe(name: str, model: Model, ingredient_encoder: LabelEncoder, measurement_encoder: LabelEncoder) -> List[str]:
    """
    
    """
    
    predicted_ingredients = model.predict([name])
    
    recipe = []
    for i in range(6):
        ingredient = predicted_ingredients[2*i]
        measurement = predicted_ingredients[2*i + 1]
        
        ingredient = np.argmax(ingredient)
        measurement = np.argmax(measurement)
        
        ingredient = ingredient_encoder.classes_[ingredient]
        measurement = measurement_encoder.classes_[measurement]
        
        if str(ingredient).strip().lower() == "nan":
            continue
        else:
            recipe.append(f"{measurement} of {ingredient}")
    
    return recipe

In [6]:
data, ingredient_encoder, measurement_encoder = data_processing("mr-boston-flattened.csv")
model = prepare_model(data, ingredient_encoder, measurement_encoder)
fit_model(model, data, ingredient_encoder, measurement_encoder)

Found 20000 different words!
Epoch 1/100
31/31 [==============================] - 6s 57ms/step - loss: 55.0220 - i1_output_loss: 6.0938 - m1_output_loss: 4.1324 - i2_output_loss: 6.2395 - m2_output_loss: 4.0407 - i3_output_loss: 6.0911 - m3_output_loss: 3.9345 - i4_output_loss: 5.7714 - m4_output_loss: 3.8520 - i5_output_loss: 4.7922 - m5_output_loss: 3.3163 - i6_output_loss: 4.2557 - m6_output_loss: 2.5024 - i1_output_acc: 0.0404 - m1_output_acc: 0.0929 - i2_output_acc: 0.0283 - m2_output_acc: 0.1444 - i3_output_acc: 0.0253 - m3_output_acc: 0.1212 - i4_output_acc: 0.0909 - m4_output_acc: 0.1444 - i5_output_acc: 0.4919 - m5_output_acc: 0.4323 - i6_output_acc: 0.7051 - m6_output_acc: 0.7626
Epoch 2/100
31/31 [==============================] - 2s 58ms/step - loss: 35.2072 - i1_output_loss: 4.7908 - m1_output_loss: 2.5731 - i2_output_loss: 5.1358 - m2_output_loss: 2.3780 - i3_output_loss: 4.8822 - m3_output_loss: 2.5275 - i4_output_loss: 3.8203 - m4_output_loss: 2.5123 - i5_output_loss: 2

31/31 [==============================] - 2s 58ms/step - loss: 28.9647 - i1_output_loss: 4.2343 - m1_output_loss: 1.9964 - i2_output_loss: 4.6621 - m2_output_loss: 1.9417 - i3_output_loss: 4.4018 - m3_output_loss: 1.9198 - i4_output_loss: 3.1905 - m4_output_loss: 1.8768 - i5_output_loss: 1.9560 - m5_output_loss: 1.3270 - i6_output_loss: 1.0911 - m6_output_loss: 0.3673 - i1_output_acc: 0.1202 - m1_output_acc: 0.2960 - i2_output_acc: 0.0879 - m2_output_acc: 0.2798 - i3_output_acc: 0.1111 - m3_output_acc: 0.3303 - i4_output_acc: 0.4081 - m4_output_acc: 0.4424 - i5_output_acc: 0.6758 - m5_output_acc: 0.6828 - i6_output_acc: 0.8394 - m6_output_acc: 0.9283 1s - loss: 28.9712 - i1_output_loss: 4.2020 - m1_output_loss: 1.9938 - i2_output_loss: 4.6181 - m2_output_loss: 1.8528 - i3_output_loss: 4.2311 - m3_output_loss: 1.8798 - i4_output_loss: 3.2799 - m4_output_loss: 1.9391 - i5_output_loss: 2.1207 - m5_output_loss: 1.4404 - i6_output_loss: 1.0449 - m6_output_loss: 0.3685 - i1_output_acc: 0.1295

31/31 [==============================] - 2s 60ms/step - loss: 24.3541 - i1_output_loss: 3.5616 - m1_output_loss: 1.6745 - i2_output_loss: 3.9441 - m2_output_loss: 1.6666 - i3_output_loss: 3.7106 - m3_output_loss: 1.6453 - i4_output_loss: 2.6672 - m4_output_loss: 1.5694 - i5_output_loss: 1.6169 - m5_output_loss: 1.0863 - i6_output_loss: 0.8781 - m6_output_loss: 0.3335 - i1_output_acc: 0.1798 - m1_output_acc: 0.4131 - i2_output_acc: 0.1515 - m2_output_acc: 0.3808 - i3_output_acc: 0.1586 - m3_output_acc: 0.4283 - i4_output_acc: 0.4253 - m4_output_acc: 0.5152 - i5_output_acc: 0.6717 - m5_output_acc: 0.7121 - i6_output_acc: 0.8384 - m6_output_acc: 0.9212
Epoch 24/100
31/31 [==============================] - 2s 59ms/step - loss: 24.0069 - i1_output_loss: 3.5369 - m1_output_loss: 1.6935 - i2_output_loss: 3.8811 - m2_output_loss: 1.6039 - i3_output_loss: 3.6358 - m3_output_loss: 1.6018 - i4_output_loss: 2.6108 - m4_output_loss: 1.5719 - i5_output_loss: 1.5747 - m5_output_loss: 1.0797 - i6_outp

31/31 [==============================] - 2s 61ms/step - loss: 20.1691 - i1_output_loss: 2.9316 - m1_output_loss: 1.4522 - i2_output_loss: 3.1931 - m2_output_loss: 1.4440 - i3_output_loss: 3.0443 - m3_output_loss: 1.4318 - i4_output_loss: 2.1366 - m4_output_loss: 1.3472 - i5_output_loss: 1.2397 - m5_output_loss: 0.9030 - i6_output_loss: 0.6906 - m6_output_loss: 0.3549 - i1_output_acc: 0.2596 - m1_output_acc: 0.5030 - i2_output_acc: 0.2242 - m2_output_acc: 0.4758 - i3_output_acc: 0.2434 - m3_output_acc: 0.4960 - i4_output_acc: 0.4879 - m4_output_acc: 0.5586 - i5_output_acc: 0.7111 - m5_output_acc: 0.7535 - i6_output_acc: 0.8626 - m6_output_acc: 0.9141
Epoch 35/100
31/31 [==============================] - 2s 59ms/step - loss: 19.9013 - i1_output_loss: 2.8390 - m1_output_loss: 1.4521 - i2_output_loss: 3.1474 - m2_output_loss: 1.4397 - i3_output_loss: 2.9788 - m3_output_loss: 1.4303 - i4_output_loss: 2.1365 - m4_output_loss: 1.3780 - i5_output_loss: 1.2360 - m5_output_loss: 0.9252 - i6_outp

31/31 [==============================] - 2s 62ms/step - loss: 17.1328 - i1_output_loss: 2.3638 - m1_output_loss: 1.2987 - i2_output_loss: 2.6391 - m2_output_loss: 1.3631 - i3_output_loss: 2.5296 - m3_output_loss: 1.2901 - i4_output_loss: 1.7674 - m4_output_loss: 1.2236 - i5_output_loss: 1.0222 - m5_output_loss: 0.8092 - i6_output_loss: 0.5482 - m6_output_loss: 0.2778 - i1_output_acc: 0.3636 - m1_output_acc: 0.5444 - i2_output_acc: 0.3101 - m2_output_acc: 0.5061 - i3_output_acc: 0.3596 - m3_output_acc: 0.5364 - i4_output_acc: 0.5596 - m4_output_acc: 0.5778 - i5_output_acc: 0.7505 - m5_output_acc: 0.7707 - i6_output_acc: 0.8758 - m6_output_acc: 0.9253
Epoch 46/100
31/31 [==============================] - 2s 61ms/step - loss: 16.8810 - i1_output_loss: 2.3721 - m1_output_loss: 1.2959 - i2_output_loss: 2.6279 - m2_output_loss: 1.3344 - i3_output_loss: 2.4601 - m3_output_loss: 1.2480 - i4_output_loss: 1.8096 - m4_output_loss: 1.1904 - i5_output_loss: 0.9660 - m5_output_loss: 0.8051 - i6_outp

31/31 [==============================] - 2s 59ms/step - loss: 14.8787 - i1_output_loss: 2.0485 - m1_output_loss: 1.1339 - i2_output_loss: 2.2304 - m2_output_loss: 1.2219 - i3_output_loss: 2.1536 - m3_output_loss: 1.2055 - i4_output_loss: 1.5504 - m4_output_loss: 1.0558 - i5_output_loss: 0.8523 - m5_output_loss: 0.7540 - i6_output_loss: 0.4335 - m6_output_loss: 0.2388 - i1_output_acc: 0.4535 - m1_output_acc: 0.6020 - i2_output_acc: 0.4081 - m2_output_acc: 0.5455 - i3_output_acc: 0.4455 - m3_output_acc: 0.5556 - i4_output_acc: 0.5869 - m4_output_acc: 0.6535 - i5_output_acc: 0.7909 - m5_output_acc: 0.7768 - i6_output_acc: 0.8859 - m6_output_acc: 0.9323
Epoch 58/100
31/31 [==============================] - 2s 64ms/step - loss: 14.8033 - i1_output_loss: 2.0197 - m1_output_loss: 1.1576 - i2_output_loss: 2.2272 - m2_output_loss: 1.2238 - i3_output_loss: 2.1393 - m3_output_loss: 1.1953 - i4_output_loss: 1.4790 - m4_output_loss: 1.0603 - i5_output_loss: 0.8443 - m5_output_loss: 0.7310 - i6_outp

31/31 [==============================] - 2s 70ms/step - loss: 13.6554 - i1_output_loss: 1.8304 - m1_output_loss: 1.0235 - i2_output_loss: 1.9504 - m2_output_loss: 1.1818 - i3_output_loss: 1.9469 - m3_output_loss: 1.1402 - i4_output_loss: 1.4090 - m4_output_loss: 0.9917 - i5_output_loss: 0.7799 - m5_output_loss: 0.7333 - i6_output_loss: 0.4264 - m6_output_loss: 0.2420 - i1_output_acc: 0.4788 - m1_output_acc: 0.6253 - i2_output_acc: 0.4747 - m2_output_acc: 0.5586 - i3_output_acc: 0.4909 - m3_output_acc: 0.5737 - i4_output_acc: 0.6343 - m4_output_acc: 0.6737 - i5_output_acc: 0.8010 - m5_output_acc: 0.7828 - i6_output_acc: 0.8899 - m6_output_acc: 0.9273
Epoch 70/100
31/31 [==============================] - 2s 74ms/step - loss: 13.5037 - i1_output_loss: 1.8334 - m1_output_loss: 1.0597 - i2_output_loss: 1.9539 - m2_output_loss: 1.1169 - i3_output_loss: 1.9706 - m3_output_loss: 1.1215 - i4_output_loss: 1.3466 - m4_output_loss: 0.9726 - i5_output_loss: 0.7450 - m5_output_loss: 0.7145 - i6_outp

31/31 [==============================] - 2s 67ms/step - loss: 12.5103 - i1_output_loss: 1.6572 - m1_output_loss: 1.0338 - i2_output_loss: 1.7641 - m2_output_loss: 1.0612 - i3_output_loss: 1.8151 - m3_output_loss: 1.0858 - i4_output_loss: 1.2766 - m4_output_loss: 0.9312 - i5_output_loss: 0.6537 - m5_output_loss: 0.6324 - i6_output_loss: 0.3661 - m6_output_loss: 0.2331 - i1_output_acc: 0.5424 - m1_output_acc: 0.6333 - i2_output_acc: 0.5202 - m2_output_acc: 0.6141 - i3_output_acc: 0.5030 - m3_output_acc: 0.5970 - i4_output_acc: 0.6687 - m4_output_acc: 0.6606 - i5_output_acc: 0.8424 - m5_output_acc: 0.8131 - i6_output_acc: 0.9152 - m6_output_acc: 0.9354
Epoch 82/100
31/31 [==============================] - 2s 62ms/step - loss: 12.2380 - i1_output_loss: 1.6242 - m1_output_loss: 0.9506 - i2_output_loss: 1.7406 - m2_output_loss: 1.1052 - i3_output_loss: 1.7034 - m3_output_loss: 1.0368 - i4_output_loss: 1.2131 - m4_output_loss: 0.9258 - i5_output_loss: 0.7002 - m5_output_loss: 0.6388 - i6_outp

31/31 [==============================] - 2s 62ms/step - loss: 11.7176 - i1_output_loss: 1.5443 - m1_output_loss: 0.9603 - i2_output_loss: 1.6373 - m2_output_loss: 1.0170 - i3_output_loss: 1.6278 - m3_output_loss: 1.0099 - i4_output_loss: 1.1791 - m4_output_loss: 0.8513 - i5_output_loss: 0.6573 - m5_output_loss: 0.6409 - i6_output_loss: 0.3834 - m6_output_loss: 0.2088 - i1_output_acc: 0.5667 - m1_output_acc: 0.6626 - i2_output_acc: 0.5758 - m2_output_acc: 0.6253 - i3_output_acc: 0.5677 - m3_output_acc: 0.6091 - i4_output_acc: 0.6949 - m4_output_acc: 0.6939 - i5_output_acc: 0.8354 - m5_output_acc: 0.8020 - i6_output_acc: 0.9020 - m6_output_acc: 0.9414
Epoch 94/100
31/31 [==============================] - 2s 71ms/step - loss: 11.7454 - i1_output_loss: 1.5464 - m1_output_loss: 0.9413 - i2_output_loss: 1.5948 - m2_output_loss: 1.0320 - i3_output_loss: 1.6627 - m3_output_loss: 1.0076 - i4_output_loss: 1.2123 - m4_output_loss: 0.9121 - i5_output_loss: 0.6585 - m5_output_loss: 0.6218 - i6_outp

In [27]:
recipe = create_recipe("Ginger Lemonade", model, ingredient_encoder, measurement_encoder)
print(recipe)

['2 oz of  Cucumber', '1 1/2 oz of  Blanco tequila', '1 oz of  Fresh Lime Juice', '3/4 oz of  Fresh lemon juice', '1/4 oz of  Simple Syrup', 'nan of  Cranberry Juice, lemon wheel']
